In [1]:
import numpy as np
import arrayfire as af
import pylab as pl
%matplotlib inline

# Optimized plot parameters to make beautiful plots:
pl.rcParams['figure.figsize']  = 9, 4
pl.rcParams['figure.dpi']      = 300
pl.rcParams['image.cmap']      = 'jet'
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 30
pl.rcParams['font.sans-serif'] = 'serif'
pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'medium'

pl.rcParams['xtick.major.size'] = 8
pl.rcParams['xtick.minor.size'] = 4
pl.rcParams['xtick.major.pad']  = 8
pl.rcParams['xtick.minor.pad']  = 8
pl.rcParams['xtick.color']      = 'k'
pl.rcParams['xtick.labelsize']  = 'medium'
pl.rcParams['xtick.direction']  = 'in'

pl.rcParams['ytick.major.size'] = 8
pl.rcParams['ytick.minor.size'] = 4
pl.rcParams['ytick.major.pad']  = 8
pl.rcParams['ytick.minor.pad']  = 8
pl.rcParams['ytick.color']      = 'k'
pl.rcParams['ytick.labelsize']  = 'medium'
pl.rcParams['ytick.direction']  = 'in'

In [2]:
# Constants Involved:
# Mass:
m   = 1
# Boltzmann Constant:
k   = 1
# Charge:
e   = -1
# Permittivity:
eps = 1

In [3]:
# Declaring the domain:
q1_start = -2 * np.pi
q1_end   =  2 * np.pi
N_q1     = 128
N_g      = 3
dq1      = (q1_end - q1_start) / N_q1

q1_center = q1_start + (0.5 + np.arange(-N_g, N_q1 + N_g)) * dq1

p1_start = -15
p1_end   =  15
N_p1     = 128
dp1      = (p1_end - p1_start) / N_p1

p1_left   = p1_start + np.arange(N_p1) * dp1
p1_center = p1_start + (0.5 + np.arange(N_p1)) * dp1
p1_right  = p1_start + (1 + np.arange(N_p1)) * dp1

# All declared arrays will follow the format (q1, p1):
p1_center, _         = np.meshgrid(p1_center, q1_center)
p1_left,   _         = np.meshgrid(p1_left,   q1_center)
p1_right,  q1_center = np.meshgrid(p1_right,  q1_center)

q1_center = af.to_array(q1_center)
p1_center = af.to_array(p1_center)
p1_left   = af.to_array(p1_left)
p1_right  = af.to_array(p1_right)

In [4]:
# Functions used for initializations:
def initialize_f(q1, v1):

    # Background Density:
    n_b = 1
    # Background Temperature:
    T_b = 1
    
    # Amplitude of Perturbation:
    alpha = 0.0001

    n   = n_b + alpha * af.cos(0.5 * q1)
    T   = T_b
    v_b = 0 #alpha * af.sin(0.5 * q1)

    f = n * (m / (2 * np.pi * k * T))**(1 / 2) \
          * af.exp(-m * (v1 - v_b)**2 / (2 * k * T)) \
    
    af.eval(f)
    return (f)

def initialize_E1(q1):
    
    # Amplitude of Perturbation:
    alpha  = 0.0001
    
    phi      = 4 * e *  alpha * af.cos(0.5 * q1)
    dphi_dq1 = (phi - af.shift(phi, 1)) / dq1
    E1       = -dphi_dq1

    af.eval(E1)
    return(E1)

In [ ]:
# Initializations:
f_n            = initialize_f(q1_center, p1_center)
f_n_plus_half  = initialize_f(q1_center, p1_center)
E1             = initialize_E1(q1_center)
E1_n_plus_half = initialize_E1(q1_center)

# Declaring the time array:
dt      = 0.00001
t_final = 0.001

time_array = np.arange(dt, t_final + dt, dt)

In [ ]:
# Used for evolution of E1:
def evolve_E1(J1):
    global E1, E1_n_plus_half
    
    E1_old = E1.copy()
    E1    += -af.tile(J1, 1, E1.shape[1]) * dt / eps

    E1_n_plus_half = 0.5 * (E1 + E1_old)
    # DEBUGGING:
    print(af.sum(af.abs((E1**2 - E1_old**2)/dt + af.tile(J1, 1, E1.shape[1]) * 2 * E1_n_plus_half)))
    print(af.sum(af.abs((af.shift(E1, -1)**2 - af.shift(E1_old, -1)**2)/dt + af.tile(af.shift(J1, -1), 1, E1.shape[1]) * 2 * af.shift(E1_n_plus_half, -1))))
    # print('J1.E1', af.sum((af.tile(J1, 1, E1.shape[1]) * E1_n_plus_half)[N_g:-N_g]))

In [ ]:
# Used for reconstruction of the distribution function:
def minmod(x, y, z):

    min_of_all = af.minof(af.minof(af.abs(x),af.abs(y)), af.abs(z))

    # af.sign(x) = 1 for x<0 and af.sign(x) = 0 for x>0:
    signx = 1 - 2 * af.sign(x)
    signy = 1 - 2 * af.sign(y)
    signz = 1 - 2 * af.sign(z)
    
    result = 0.25 * af.abs(signx + signy) * (signx + signz) * min_of_all

    af.eval(result)
    return result

def slope_minmod(input_array, axis):

    if(axis == 0):
        
        f_i_plus_one  = af.shift(input_array, -1)
        f_i_minus_one = af.shift(input_array,  1)
  
    elif(axis == 1):
        
        f_i_plus_one  = af.shift(input_array, 0, -1)
        f_i_minus_one = af.shift(input_array, 0,  1)

    forward_diff  = (f_i_plus_one - input_array  )
    backward_diff = (input_array  - f_i_minus_one)
    central_diff  = backward_diff + forward_diff

    slope_lim_theta = 2

    left   = slope_lim_theta * backward_diff
    center = 0.5 * central_diff
    right  = slope_lim_theta * forward_diff

    return(minmod(left, center, right))

def reconstruct_minmod(variable_to_reconstruct, axis):
    
    slope = slope_minmod(variable_to_reconstruct, axis)

    left_face_value  = variable_to_reconstruct - 0.5 * slope
    right_face_value = variable_to_reconstruct + 0.5 * slope
   
    return(left_face_value, right_face_value)

In [ ]:
# Riemann solver used:
def riemann_solver(left_state, right_state, velocity):
    upwind_state = af.select(velocity > 0, 
                             left_state,
                             right_state
                            )

    af.eval(upwind_state)
    return(upwind_state)

In [ ]:
# Used for the evolution of the distribution function(s):
# When at_n is toggled to true, it returns the df_dt for the evolution
# equation used by f^n:
def df_dt_fvm(f, at_n):
    # Initializing df_dt:
    df_dt = 0
    
    f_left_plus_eps, f_right_minus_eps = reconstruct_minmod(f, 0)
    # f_left_minus_eps of i-th cell is f_right_minus_eps of the (i-1)th cell
    f_left_minus_eps = af.shift(f_right_minus_eps, 1)
    
    if(at_n == True):
        C_q1 = p1_center**3 / 2
    else:
        C_q1 = p1_center

    f_left  = riemann_solver(f_left_minus_eps, f_left_plus_eps, C_q1)
    f_right = af.shift(f_left, -1)
    
    left_flux  = C_q1 * f_left
    right_flux = C_q1 * f_right

    df_dt += -0*(right_flux - left_flux) / dq1
    
    global E1, E1_n_plus_half
    # Source term for the f_n evolution equation:
    if(at_n):
        print('1', af.sum(af.abs(af.sum(df_dt, 1))))
        J1 = e * af.sum(f_left * p1_center, 1) * dp1
        evolve_E1(J1)
        
        J1_left  = J1
        J1_right = af.shift(J1, -1)
        
        E1_left  = E1_n_plus_half
        E1_right = af.shift(E1_left, -1)
        df_dt   += 0.5 * (e / m) * p1_center * (f_left * E1_left + f_right * E1_right)
    
        print('2', af.sum(df_dt) - af.sum(0.5 * (e / m) * p1_center * (f_left * E1_left + f_right * E1_right)))
        dE_dt = af.sum(df_dt, 1) * dp1
        print('3', af.sum(af.abs(J1_left * E1_left[:, 0] - af.sum(e * f_left * E1_left * p1_center, 1) * dp1)))
        print('4', af.sum(af.abs(J1_right * E1_right[:, 0] - af.sum(e * f_right * E1_right * p1_center, 1) * dp1)))
        print('5', af.sum(af.abs(  J1_left * E1_left[:, 0] + J1_right * E1_right[:, 0] 
                                 - af.sum(  e * f_left * E1_left * p1_center 
                                          + e * f_right * E1_right * p1_center, 1) * dp1
                                )
                         )
             )
        print('6', af.sum(af.abs(  2 * af.sum(df_dt, 1) * dp1 
                                 - af.sum(  e * f_left * E1_left * p1_center 
                                          + e * f_right * E1_right * p1_center, 1) * dp1
                                )
                         )
             )
        
        print('dE_dt', af.sum(af.abs((dE_dt - 0.5 * (J1_left * E1_left[:, 0] + J1_right * E1_right[:, 0]))[N_g:-N_g])))

    
    if(at_n == True):
        # Getting C_p at q1_left_p1_center:
        C_p1_center = 0.5 * (e / m) * E1_n_plus_half * p1_center**2
        # Getting C_p at q1_left_p1_left:
        C_p1_left   = 0.5 * (e / m) * E1_n_plus_half * p1_left**2
        # Getting C_p at q1_left_p1_right:
        C_p1_right  = 0.5 * (e / m) * E1_n_plus_half * p1_right**2

        # Variation of p1 is along axis 0:
        f_p1_left_plus_eps, f_p1_right_minus_eps = reconstruct_minmod(f, 1)
        # f_left_minus_eps of i-th cell is f_right_minus_eps of the (i-1)th cell
        f_p1_left_minus_eps = af.shift(f_p1_right_minus_eps, 0, 1)
        f_p1_left  = riemann_solver(f_p1_left_minus_eps, f_p1_left_plus_eps, C_p1_center)
        f_p1_right = af.shift(f_p1_left, 0, -1)
        
        # For flux along p1 at q1_left:
        flux_p1_left_at_q1_left  = C_p1_left  * f_p1_left
        flux_p1_right_at_q1_left = C_p1_right * f_p1_right

        d_flux_p1_at_q1_left  = flux_p1_right_at_q1_left - flux_p1_left_at_q1_left
        d_flux_p1_at_q1_right = af.shift(d_flux_p1_at_q1_left, -1)

        df_dt += -0.5 * (d_flux_p1_at_q1_left + d_flux_p1_at_q1_right) / dp1
        af.eval(df_dt)

    else:
        # Getting C_p at q1_left_p1_center:
        C_p1 = (e / m) * E1
        # Variation of p1 is along axis 0:
        f_p1_left_plus_eps, f_p1_right_minus_eps = reconstruct_minmod(f, 1)
        # f_left_minus_eps of i-th cell is f_right_minus_eps of the (i-1)th cell
        f_p1_left_minus_eps = af.shift(f_p1_right_minus_eps, 0, 1)
        f_p1_left = riemann_solver(f_p1_left_minus_eps, f_p1_left_plus_eps, C_p1)

        # For flux along p1 at q1_left:
        flux_p1_left_at_q1_left  = C_p1 * f_p1_left
        flux_p1_right_at_q1_left = af.shift(flux_p1_left_at_q1_left, 0, -1)

        d_flux_p1_at_q1_left  = flux_p1_right_at_q1_left - flux_p1_left_at_q1_left
        d_flux_p1_at_q1_right = af.shift(d_flux_p1_at_q1_left, -1)

        df_dt += -0.5 * (d_flux_p1_at_q1_left + d_flux_p1_at_q1_right) / dp1
        af.eval(df_dt)
    
    # DEBUGGING:
    import time
    time.sleep(1)
#     if(at_n):

#         J1_l = e * af.sum(f_left * p1_center, 1) * dp1
#         J1_r = e * af.sum(f_right * p1_center, 1) * dp1
#         E1_l = E1_n_plus_half[:, 0]
#         E1_r = af.shift(E1_n_plus_half[:, 0], -1)
        
        
#         print('dE_dt', af.sum((dE_dt - 0.5 * (J1_l * E1_l + J1_r * E1_r))[N_g:-N_g]))
#     else:
#         print('dn_dt', af.sum(df_dt[N_g:-N_g]))

    if(at_n):
        return(2 * df_dt / p1_center**2)
    else:
        return df_dt

In [ ]:
def timestep():
    
    global f_n, f_n_plus_half
    # Applying periodic boundary conditions:
    f_n[:N_g]  = f_n[-2 * N_g:-N_g]
    f_n[-N_g:] = f_n[N_g:2*N_g]
    
    # DEBUGGING:
    # print('f_n')
    f_n_plus_half = f_n_plus_half + df_dt_fvm(f_n, False) * dt
    af.eval(f_n_plus_half)

    # Applying periodic boundary conditions:
    f_n_plus_half[:N_g]  = f_n_plus_half[-2 * N_g:-N_g]
    f_n_plus_half[-N_g:] = f_n_plus_half[N_g:2*N_g]
    # DEBUGGING:
    # print('f_n_plus_half')
    f_n = f_n + df_dt_fvm(f_n_plus_half, True) * dt
    af.eval(f_n)

In [ ]:
# Main Loop:
initial_mass            = af.sum(f_n_plus_half[N_g:-N_g]) * dp1
initial_kinetic_energy  = 0.5 * m * af.sum((f_n * p1_center**2)[N_g:-N_g]) * dp1
initial_electric_energy = 0.25 * eps * (af.sum(E1[N_g:-N_g])**2 + af.sum(af.shift(E1, -1)[N_g:-N_g])**2)
initial_total_energy    = initial_kinetic_energy + initial_electric_energy

n_data = np.zeros(time_array.size)
for time_index, t0 in enumerate(time_array):
    n_data[time_index] = af.max(af.sum(f_n[N_g:-N_g], 1) * dp1)
    if((time_index + 1)%100 == 0):
        print('Computing For Time = %.3f'%t0)
    timestep()

1 0.0
3.89934509665362e-15
3.8993450966536195e-15
2 0.0
3 2.2794533845491193e-19
4 2.2794533845491193e-19
5 3.7758188630985913e-19
6 4.391677647924191e-19
dE_dt 2.1213782643228656e-19
1 0.0
3.4116775395150855e-15
3.4116775395150855e-15
2 0.0
3 2.0902387664756073e-19
4 2.0902387664756073e-19
5 4.04470201662578e-19
6 4.629626490882727e-19
dE_dt 2.133128837262537e-19
1 0.0
3.876969383675423e-15
3.876969383675423e-15
2 0.0
3 2.219979781463878e-19
4 2.219979781463878e-19
5 3.725025247201271e-19
6 4.485076678839085e-19
dE_dt 2.0920515773372665e-19
1 0.0
3.5037388551865568e-15
3.5037388551865568e-15
2 0.0
3 2.4741107952905834e-19
4 2.4741107952905834e-19
5 3.938324323296308e-19
6 4.454305560052113e-19
dE_dt 2.268342967914264e-19
1 0.0
3.3328233836723114e-15
3.3328233836723114e-15
2 0.0
3 2.494146933901021e-19
4 2.494146933901021e-19
5 4.1463700120630057e-19
6 3.890747408719735e-19
dE_dt 2.290657512817179e-19
1 0.0
3.505192173484312e-15
3.505192173484312e-15
2 0.0
3 2.5480610595356866e-19
4 2.

In [ ]:
final_mass            = af.sum(f_n_plus_half[N_g:-N_g]) * dp1
final_kinetic_energy  = 0.5 * m * af.sum((f_n * p1_center**2)[N_g:-N_g]) * dp1
final_electric_energy = 0.25 * eps * (af.sum(E1[N_g:-N_g])**2 + af.sum(af.shift(E1, -1)[N_g:-N_g])**2)
final_total_energy    = final_kinetic_energy + final_electric_energy

In [ ]:
abs(final_mass - initial_mass)

In [ ]:
abs(final_kinetic_energy - initial_kinetic_energy)

In [ ]:
abs(final_electric_energy - initial_electric_energy)

In [ ]:
abs(final_total_energy - initial_total_energy)

In [ ]:
pl.plot(time_array, n_data)